## Revtsov Midterm Project (Part 2)

### Code to Retrieve, Process Data

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import cvxpy as cp
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import itertools
import portfolio_optimization as opt

(CVXPY) May 01 08:31:51 AM: Encountered unexpected exception importing solver OSQP:
ImportError('DLL load failed while importing qdldl: The specified module could not be found.')


<style>
    .hide_code {
        display: none;
    }
</style>

In [2]:
universe = pd.DataFrame(
    data=[
        ['Large Cap US Equity', 'IWB'],
        ['Small Cap US Equity', 'IWM'],
        ['Dev Mkts non-US Equity', 'EFA'],
        ['Emerg Mkts Equity', 'EEM'],
        ['Global REIT', 'VNQ'],
        ['Corp Bonds', 'LQD'],
        ['Short-Term Treasury', 'SHY'],
    ],
    columns=['Segment', 'Ticker']
)

In [3]:
# load the data
frames = []
for _, row in universe.iterrows():
    ticker = row.Ticker
    frames.append(pd.read_csv(f'{ticker}_daily.csv').set_index('Date')['Adj Close'].rename(ticker))

prices = pd.concat(frames, axis=1)
n_obs = prices.shape[0]

In [4]:
# define frequencies we want to analyze
freqs = {
    # 'daily': 1,
    # 'weekly': 5,
    # 'monthly': 21,
    # 'quarterly': 62,
    'yearly': 252
}

In [5]:
# calculate the simple price change for each period
returns = {k: prices.iloc[0:n_obs:v, :].pct_change().dropna(axis=0, thresh=prices.shape[1]) for k,v in freqs.items()}

In [6]:
portfolio_return_targets = [0.02, 0.04, 0.06]
portfolio_return_targets = [0.06]

In [7]:
import portfolio_optimization as opt
frames = opt.run_opts(returns, portfolio_return_targets)

Running opts: 100%|██████████| 1/1 [00:00<00:00, 100.01it/s]


[     Mean-Var  Mean-MAD  Mean-CVaR
IWB  0.448528  0.440350   0.384636
IWM  0.000000 -0.000000   0.000000
EFA  0.000000 -0.000000   0.000000
EEM  0.000000  0.000000   0.000000
VNQ  0.000000  0.000000   0.000000
LQD  0.050982  0.081996   0.293290
SHY  0.500490  0.477654   0.322074]


In [8]:
```

SyntaxError: invalid syntax (1986969183.py, line 1)

In [ ]:
list(itertools.product(list(returns.keys()), portfolio_return_targets))

In [ ]:
# for freq, rts in returns.items():
freq = 'monthly'
def calc_stats(freq, returns):
    rts = returns[freq]
    period_stats = pd.DataFrame(index=None, columns=rts.columns)
    period_stats.loc['Avg Return', :] = rts.mean()
    period_stats.loc['Std Dev', :] = rts.std()
    period_stats.loc['Return/Risk', :] = period_stats.loc['Avg Return', :] / period_stats.loc['Std Dev', :]
    period_stats.loc['Skewness', :] = rts.skew()
    period_stats.loc['Kurtosis', :] = rts.kurt()
    period_stats.loc['10th Pct', :] = rts.apply(np.percentile, axis=0, q=10)
    period_stats.loc['25th Pct', :] = rts.apply(np.percentile, axis=0, q=25)
    period_stats.loc['50th Pct', :] = rts.apply(np.percentile, axis=0, q=50)
    period_stats.loc['75th Pct', :] = rts.apply(np.percentile, axis=0, q=75)
    period_stats.loc['90th Pct', :] = rts.apply(np.percentile, axis=0, q=90)
    
    stats_styled = period_stats.T.style.format({
        'Avg Return': '{:,.2%}',
        'Std Dev': '{:,.2%}',
        'Return/Risk': '{:,.2}',
        'Skewness': '{:,.2}',
        'Kurtosis': '{:,.2}',
        '10th Pct': '{:,.2%}',
        '25th Pct': '{:,.2%}',
        '50th Pct': '{:,.2%}',
        '75th Pct': '{:,.2%}',
        '90th Pct': '{:,.2%}',
    })
    return stats_styled


def plot_corr(freq, returns, lower_only=True):
    rts = returns[freq]
    cor = rts.corr()
    if lower_only:
        # create mask to only show bottom triangle of matrix
        mask = np.triu(np.ones_like(cor, dtype=bool))
        # exclude diagonal (we want to see it)
        mask[np.diag_indices_from(mask)] = False
    else:
        mask = np.zeros_like(rts.corr())
    hm = sns.heatmap(cor, mask=mask, cmap='coolwarm', center=0, annot=True)
    _ = hm.set_title(f'{freq.capitalize()} Retun Correlations')

def plot_time_series(freq, returns):
    rts = returns[freq]
    rts.add(1).cumprod().add(-1).plot(kind='line', title=f'{freq.capitalize()} Cumulative Returns')

def plot_hist(freq, returns, alpha=0.5):
    rts = returns[freq]
    fig, ax = plt.subplots()

    for col in rts.columns:
      ax.hist(rts[col], alpha=alpha, bins=50)  # Adjust alpha as needed

    ax.legend(rts.columns, loc='best', prop={'size':10}, handletextpad=0.2)  # Adjust legend options 
    
    # Add labels and title
    plt.xlabel("Return")
    plt.ylabel("Frequency")
    plt.title(f"{freq.capitalize()} Return Distribution")
    
    # Show the plot
    plt.show()

### Yearly Stats

In [ ]:
calc_stats('yearly', returns)

In [ ]:
plot_corr('yearly', returns, lower_only=False)

In [ ]:
def solve_problem(mean, cov, w, obj, constraints, labels):
    prob = cp.Problem(
        objective=obj,
        constraints=constraints,
    )
    
    prob.solve(solver=cp.ECOS)
    assert prob.status == 'optimal'
    
    p_var = w.value @ cov @ w.value.T
    p_risk = np.sqrt(p_var)
    p_ret = w.value @ mean
    print('\nPortfolio Weights:\n')
    print(pd.Series(index=labels, data=np.round(w.value * 1e2, 2), name='Weight'))
    print(f'\nPortfolio risk: {round(p_risk * 1e2, 2)}%')
    print(f'Portfolio return: {round(p_ret * 1e2, 2)}%')

### MV

In [ ]:
rts = returns['yearly'].dropna()
ann_factor = freqs['yearly']
n_assets = rts.shape[1]
mean = rts.mean()
mean

In [ ]:
# define the vector we're solving
w = cp.Variable(n_assets)

constraints = [
    # sum of all weights is one
    cp.sum(w) == 1,
    # all weights non-negative
    w >= 0,
    # set the expected return
    w @ mean >= 0.06 * (ann_factor / 252),
]

solve_min_var(mean=mean, cov=rts.cov(), w=w, constraints=constraints, labels=list(mean.index))

In [ ]:
def solve_min_var(mean, cov, w, constraints, labels):
    """
    Helper function that will be used throughout the homework
    """
    # minimize variance of portfolio
    obj = cp.Minimize(cp.quad_form(w, cov))
    
    prob = cp.Problem(
        objective=obj,
        constraints=constraints,
    )
    
    prob.solve(solver=cp.ECOS)
    assert prob.status == 'optimal'

    p_var = w.value @ cov @ w.value.T
    p_risk = np.sqrt(p_var)
    p_ret = w.value @ mean
    print('\nPortfolio Weights:\n')
    print(pd.Series(index=labels, data=np.round(w.value * 1e2, 2), name='Weight'))
    print(f'\nPortfolio risk: {round(p_risk * 1e2, 2)}%')
    print(f'Portfolio return: {round(p_ret * 1e2, 2)}%')

### Mean-MAD

In [ ]:
w = cp.Variable(n_assets)

In [ ]:
obj = cp.Minimize(cp.sum(cp.abs((rts.values @ w) - (mean.values @ w))))

constraints = [
    # sum of all weights is one
    cp.sum(w) == 1,
    # all weights non-negative
    w >= 0,
    # set the expected return
    w @ mean >= 0.04 * (ann_factor / 252),
]

solve_problem(mean=mean, cov=rts.cov(), w=w, obj=obj, constraints=constraints, labels=mean.index) 

### Mean-CVaR

In [ ]:
alpha = 10  # Confidence level for CVaR

In [ ]:
def var(xs, alpha):
    return np.percentile(xs, alpha, method='interpolated_inverted_cdf')
    # var_index = int(np.floor(len(xs) * alpha))
    # return xs.sort_values(ascending=True).iloc[var_index-1]

def cvar(xs, alpha):
    return xs[xs < var(xs, alpha)].mean()
    # var_index = int(np.floor(len(xs) * alpha))
    # return xs.sort_values(ascending=True).iloc[:var_index-1].mean()

def objective(weights, pars):
    alpha = pars[0]
    portfolio_rts = (rts @ weights)
    return cvar(portfolio_rts, alpha) * -1

# function to be used for total weight constraint
def total_constraint(x, total_weight):
    return np.sum(x) - total_weight

def target_return_constraint(x, mean, target_return):
    return (x @ mean) - target_return

# Initial guess for the weights
w0 = np.ones(n_assets) / n_assets

# Define bounds for the weights (weights should be between 0 and 1)
bounds = [(0, 1) for _ in range(n_assets)]

cons = (
    # sum of weights = 1
    {'type': 'eq', 'fun': total_constraint, 'args': [1]},
    # target return
    {'type': 'eq', 'fun': target_return_constraint, 'args': [mean.values, 0.04]},
)

# Minimize the CVaR
result = minimize(
    objective, 
    w0, 
    constraints=cons, 
    args=[alpha], 
    bounds=bounds
)

# Display results
optimal_weights = np.round(result.x, 8)
min_CVaR = result.fun
print("Optimal portfolio weights:", optimal_weights)
print("Minimum CVaR:", min_CVaR)

In [ ]:
import multiprocessing